In [ ]:
# from IPython.core.display import display, HTML
# display(HTML(d|ata="""
# <style>
#     div#notebook-container    { width: 95%; }
#     div#menubar-container     { width: 65%; }
#     div#maintoolbar-container { width: 99%; }
# </style>
# """))

# #width for some images
# im_width = 11

In [ ]:
alldata = dict()

In [ ]:
# %matplotlib nbagg
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patch
from matplotlib.patches import Polygon

plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.minor.visible'] = True

import pandas as pd

import numpy as np
import scipy as sp
# import scipy.optimize 
import scipy.optimize as opt
from scipy.optimize import curve_fit
from scipy.misc import face
from scipy import interpolate

import glob
import csv
import re
import sys 
import os
import copy
import time
import h5py
import skimage.feature
import skimage.filters
import skimage.measure
import socket  
import itertools
from tqdm.notebook import tqdm as tqdm

from skimage import filters


from IPython.display import display, clear_output, Markdown
from PIL import Image, ImageOps

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

import cv2

udir = '/cds/home/d/diegotur/UED/'

if udir not in sys.path:
    sys.path.append(udir)

import ued_dt3 as ued_dt
try:
    import pyFAI, pyFAI.detectors
    from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
except:
    print('No pyFAI library found. If you want to do azimuthal integrations - install it!')

import dt_functions as dt
import I_UED_dt


import lmfit
from lmfit import Model, Parameters

import imp

In [ ]:
import pickle

In [ ]:
# Loading binned data with static analysis done. 
with open('test_save_binned_data/saved_dif3.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

dif = loaded_dict

# Time Resolved Analysis of Bragg Peaks

## Fit Braggs over time delays

In [ ]:
dif.tmp_hk = np.zeros((120,2))
for i in np.arange(len(dif.allBragg_names_1)):
    tmp_h = 0
    counter = 0
    if dif.allBragg_names_1[i][counter] == '-':
        tmp_h = -1*int(dif.allBragg_names_1[i][counter+1])
        counter += 2
    else:
        tmp_h = int(dif.allBragg_names_1[i][counter])
        counter += 1

    if dif.allBragg_names_1[i][counter] == '-':
        tmp_k = -1*int(dif.allBragg_names_1[i][counter+1])
        counter += 2
    else:
        tmp_k = int(dif.allBragg_names_1[i][counter])
        counter += 1

    dif.tmp_hk[i] = tmp_h, tmp_k

In [ ]:
dif.allBragg_names

In [ ]:
dif.allBragg_names_1

In [ ]:
dif.tmp_hk

In [ ]:
# %%timeit
# this loop takes 2s per scattering pattern 
#set xx and yy coordinate frame 

# x = np.arange(dif.first_image.shape[0])
# y = np.arange(dif.first_image.shape[1])
# xx, yy = np.meshgrid(x,y)

image = dif.first_image

dif.centers_binned_delays = np.zeros_like(dif.allBragg_coord_2)
dif.bragg_fits_binned_delays = np.zeros((len(dif.binned_images), 120, 9 ))
dif.fit_erros_binned_delays = np.zeros((len(dif.binned_images), 120, 9,9))
# dif.fit_erros_binned_delays = np.zeros((len(dif.binned_images), 120, 6,6))


for idx, j in enumerate(np.arange(len(dif.binned_images))):
    print(idx, j)
#     image = dif.load_img(dif.fnames[sel_pre_t0[0][j:j+24]])
    image = dif.binned_images[idx] - dif.bkg_img_bkg
#     image = dif.binned_images[idx] - empty_img
        
#     image = ued_dt.despike(image)
#     image /= image[sel_corners].mean()


    for i in np.arange(len(dif.allBragg_names_2)):
        if dif.sel_bragg_in_im[i]:

            ### ROI 1: 
            roi_1_lower_left = (dif.allBragg_coord_2[i] - dif.roisize).astype(int)
            roi_1_upper_right = (dif.allBragg_coord_2[i] + dif.roisize).astype(int)
            roi_1_selx  = np.arange(roi_1_lower_left[1],roi_1_upper_right[1])
            roi_1_sely =  np.arange(roi_1_lower_left[0],roi_1_upper_right[0])
            roi_1 = image[roi_1_selx,:][:,roi_1_sely] #- empty_img[roi_1_selx,:][:,roi_1_sely]
            
            
            tmp_hist_1, tmp_bins_1= np.histogram(roi_1, bins = 100)
            tmp_hist_norm_1, tmp_bins_norm_1= np.histogram(roi_1, bins = 100, normed = True )
            tmp_bins_norm_1 = tmp_bins_norm_1[1:]
            threshold_1 = tmp_bins_norm_1[np.cumsum(tmp_hist_norm_1) * (tmp_bins_norm_1[1] - tmp_bins_norm_1[0]) > 0.99][0]

            gradient_roi_1 = filters.sobel(roi_1/np.mean(roi_1))
            gradient_roi_1_hist_1, gradient_roi_1_bins_1= np.histogram(gradient_roi_1, bins = 1000)
            gradient_roi_1_hist_norm_1, gradient_roi_1_bins_norm_1= np.histogram(gradient_roi_1, bins = 1000, normed = True )
            gradient_roi_1_bins_norm_1 = gradient_roi_1_bins_norm_1[1:]
            gradient_threshold_1 = gradient_roi_1_bins_1[1:][np.cumsum(gradient_roi_1_hist_norm_1) * (gradient_roi_1_bins_norm_1[1] - gradient_roi_1_bins_norm_1[0]) > 0.997][0]



            ### Fit ROIs with a 2D gaussian: 
#             np.linalg.norm([dif.allBragg_coord_2[i][0]-dif.centerpos[0], dif.allBragg_coord_2[i][1]-dif.centerpos[1]])*coef
#             a0_tmp = 150/(np.linalg.norm([dif.allBragg_coord_2[i][0]-dif.centerpos[0], dif.allBragg_coord_2[i][1]-dif.centerpos[1]])*coef)**2
#             initial_guess_1 = (a0_tmp,dif.allBragg_coord_2[i][0],dif.allBragg_coord_2[i][1],7.5,7.5,0,-0.25)
            initial_guess_1 = dif.static.bragg_fits3[0, i, :]
            xx_yy_1 = (dif.xx[roi_1_selx,:][:,roi_1_sely], dif.yy[roi_1_selx,:][:,roi_1_sely])
        
            params_bragg_t0_1 = Parameters()
            # (amp0_1, x0_1, y0_1, sigmax_1, sigmay_1,theta0_1, offset0_1, initial_ax_1, initial_by_1)
            params_bragg_t0_1.add('amplitude', value=initial_guess_1[0])
            params_bragg_t0_1.add('xo', value=initial_guess_1[1])
            params_bragg_t0_1.add('yo', value=initial_guess_1[2])
            params_bragg_t0_1.add('sigma_x', value=initial_guess_1[3], vary = False)
            params_bragg_t0_1.add('sigma_y', value=initial_guess_1[4], vary = False)
            params_bragg_t0_1.add('theta', value=initial_guess_1[5], vary = False)
            params_bragg_t0_1.add('offset', value=initial_guess_1[6])
            params_bragg_t0_1.add('ax', value=initial_guess_1[7])
            params_bragg_t0_1.add('by', value=initial_guess_1[8])
            
            result_tmp_1 = dif.dmodel_bragg.fit(roi_1.ravel(), params_bragg_t0_1, xx_yy=xx_yy_1)
#             print(result_tmp.best_values)
            pcov_1 = result_tmp_1.covar
            popt_1 = result_tmp_1.best_values['amplitude'], result_tmp_1.best_values['xo'], \
        result_tmp_1.best_values['yo'], result_tmp_1.best_values['sigma_x'], result_tmp_1.best_values['sigma_y'], result_tmp_1.best_values['theta'], result_tmp_1.best_values['offset'], result_tmp_1.best_values['ax'], result_tmp_1.best_values['by'] 
        
        
#             popt_1, pcov_1 = opt.curve_fit(ued_dt.Gaussian_2D1_with_plane, xx_yy_1, roi_1.ravel(), p0=initial_guess_1,maxfev=5000 )
        
#             try:
#                 popt_1, pcov_1 = opt.curve_fit(ued_dt.Gaussian_2D1_with_plane, xx_yy_1, roi_1.ravel(), p0=initial_guess_1,maxfev=5000 )
#             except:
#                 print('spike !?')
#                 popt_1, pcov_1 = popt_1*np.nan, pcov_1*np.nan
                
            dif.centers_binned_delays[i]  = popt_1[1], popt_1[2]
            dif.bragg_fits_binned_delays[idx][i] =popt_1
            dif.fit_erros_binned_delays[idx][i][0:3, 0:3] = pcov_1[0:3, 0:3]
            dif.fit_erros_binned_delays[idx][i][6:9, 6:9] = pcov_1[3:6, 3:6]
            if gradient_threshold_1> 17 :
                print(f"big ass spike detected in time bin{idx} and peak nb {i}. which corresponds to {dif.allBragg_indices_2[i]}")
#                 dif.centers_binned_delays[i]  = popt_1[1]*np.nan, popt_1[2]*np.nan
#                 dif.bragg_fits_binned_delays[idx][i] =popt_1*np.nan
#                 dif.fit_erros_binned_delays[idx][i] = pcov_1*np.nan                

        else:
            pass


In [ ]:
plt.figure()
plt.imshow(roi_1)#, vmin = 0.63, vmax=0.80)
plt.colorbar()

In [ ]:
plt.figure(figsize = (8,8))
# plt.imshow(dif.first_image-dif.bkg_img_bkg, vmin = -0.01, vmax=2, cmap='binary')

im_color = copy.copy(dif.first_image-dif.bkg_img_bkg)

# x = np.arange(image.shape[0])
# y = np.arange(image.shape[1])
# x, y = np.meshgrid(x, y)

sel_corners = (
    (dif.y< ((100-0)/100)*dif.x - 900) + (dif.y< ((-100)/100)*dif.x +150) + 
    (dif.y> ((100-0)/100)*dif.x + 900) + (dif.y> ((-100)/100)*dif.x +900+900+140)
)

#             image /= image[sel_corners].mean()
image /= image[~sel_corners].mean()
im_color[sel_corners] = 1000
 
# im_color[]
plt.imshow(im_color, vmin = -0.1, vmax=0.1, cmap='binary')

# plt.scatter(dif.bragg_fits_binned_delays[0,:,1], dif.bragg_fits_binned_delays[0,:,2])

### Check how the center of the image shifts ... 

In [ ]:
# errors to standard deviation

dif.fit_erros_binned_delays_sigma= np.zeros((dif.fit_erros_binned_delays.shape[0:-1]))
for idx, j in enumerate(np.arange(len(dif.binned_images))):
    for i in np.arange(len(dif.allBragg_names_1)):
        if dif.sel_bragg_in_im[i]:
            dif.fit_erros_binned_delays_sigma[idx][i] = np.sqrt(np.diag(dif.fit_erros_binned_delays[idx][i]))

dif.error_fit_all_I_I0 = np.abs(dif.bragg_fits_binned_delays / dif.bragg_fits_binned_delays[0]) * np.sqrt(
    
    ((dif.fit_erros_binned_delays_sigma / dif.bragg_fits_binned_delays)**2) +  
    (dif.fit_erros_binned_delays_sigma[0] / dif.bragg_fits_binned_delays[0])**2 )


In [ ]:
dif.centers_time = np.zeros((len(dif.bincenters), 2))
dif.centers_time_errors = np.zeros((len(dif.bincenters), 2))

for t_idx in range(len(dif.bincenters)):
    
    center_good_bragg_mask = copy.copy(dif.sel_bragg_in_im)
    
    centers_tmp_x = 0.5*(dif.bragg_fits_binned_delays[t_idx, :, 1][::-1] + dif.bragg_fits_binned_delays[t_idx, :, 1])
    centers_tmp_x2 = centers_tmp_x[center_good_bragg_mask]
    
    errors_tmp_x = np.sqrt( 0.25*dif.fit_erros_binned_delays_sigma[t_idx, :, 1][::-1]**2  + 0.25*dif.fit_erros_binned_delays_sigma[t_idx, :, 1]**2 )
    errors_tmp_x2 = errors_tmp_x[center_good_bragg_mask]
    

    centers_tmp_y = 0.5*(dif.bragg_fits_binned_delays[t_idx, :, 2][::-1] + dif.bragg_fits_binned_delays[t_idx, :, 2])
    centers_tmp_y2 = centers_tmp_y[center_good_bragg_mask]
    
    errors_tmp_y = np.sqrt( 0.25*dif.fit_erros_binned_delays_sigma[t_idx, :, 2][::-1]**2  + 0.25*dif.fit_erros_binned_delays_sigma[t_idx, :, 2]**2 )
    errors_tmp_y2 = errors_tmp_y[center_good_bragg_mask]
    
    
    center_good_bragg_mask[np.where(centers_tmp_x < np.nanmean(centers_tmp_x2)*0.9)] = False
    center_good_bragg_mask[np.where(centers_tmp_y < np.nanmean(centers_tmp_y2)*0.9)] = False

    centers_tmp_x = centers_tmp_x[center_good_bragg_mask]
    centers_tmp_y = centers_tmp_y[center_good_bragg_mask]
    
    errors_tmp_x = errors_tmp_x[center_good_bragg_mask]
    errors_tmp_y = errors_tmp_y[center_good_bragg_mask]
    
    dif.centers_time[t_idx] = np.asarray([np.nanmean(centers_tmp_x), np.nanmean(centers_tmp_y)] )
    n = sum(center_good_bragg_mask)/2
    
    dif.centers_time_errors[t_idx] = np.asarray([np.nanmean(errors_tmp_x), np.nanmean(errors_tmp_y)])/np.sqrt(n)
#     centers_time_errors[t_idx] = np.asarray([np.std(centers_tmp_x), np.std(centers_tmp_y)])



In [ ]:
# center moving over time 
plt.errorbar(dif.bincenters, dif.centers_time[:,0], yerr =  dif.centers_time_errors[:,0] )
plt.axhline(np.nanmean(dif.centers_time[:,0]))
# plt.axvline(dif.bincenters[14])
# plt.ylim(489.65,489.75)
plt.xlim(-2,8)
# plt.axvline(dif.bincenters[14], c='k')

In [ ]:
# i=59
# i=119
i=49


# i=53
# i=6
# i=48
# print(sel_bragg_in_im[i])

# j= 32
# i= np.arange(len(dif.sel_bragg_in_im))[dif.sel_bragg_in_im][j]
# i = 59
# i = 113
# t_idx = 1
# print(i)
print('Bragg in img ? ')
print(dif.sel_bragg_in_im[i])
t_idx = 14
# i=114



image = dif.binned_images[t_idx]

# image = dif.load_img(dif.fnames_binned[14][0])

roi_1_lower_left = (dif.allBragg_coord_2[i] - dif.roisize).astype(int)
roi_1_upper_right = (dif.allBragg_coord_2[i] + dif.roisize).astype(int)
roi_1_selx  = np.arange(roi_1_lower_left[1],roi_1_upper_right[1])
roi_1_sely =  np.arange(roi_1_lower_left[0],roi_1_upper_right[0])
roi_1 = image[roi_1_selx,:][:,roi_1_sely] - dif.bkg_img_bkg[roi_1_selx,:][:,roi_1_sely]

xx_yy_1 = (dif.xx[roi_1_selx,:][:,roi_1_sely], dif.yy[roi_1_selx,:][:,roi_1_sely])


xx_yy_1 = (dif.xx[roi_1_selx,:][:,roi_1_sely], dif.yy[roi_1_selx,:][:,roi_1_sely])
tmp_hist_1, tmp_bins_1= np.histogram(roi_1, bins = 100)
tmp_hist_norm_1, tmp_bins_norm_1= np.histogram(roi_1, bins = 100, normed = True )
tmp_bins_norm_1 = tmp_bins_norm_1[1:]
threshold_1 = tmp_bins_norm_1[np.cumsum(tmp_hist_norm_1) * (tmp_bins_norm_1[1] - tmp_bins_norm_1[0]) > 0.99][0]
mnorm2d_1 = np.ma.masked_less(roi_1,threshold_1)

com_1 = sp.ndimage.center_of_mass(mnorm2d_1)


offset0_1 = np.nanmean(tmp_bins_1[1:][5:10])
amp0_1 = np.nanmean(tmp_bins_1[1:][80:90]) - np.nanmean(tmp_bins_1[1:][5:10]) # amplitude
x0_1 = xx_yy_1[0][int(com_1[0]), int(com_1[1])] 
y0_1 = xx_yy_1[1][int(com_1[0]), int(com_1[1])]
sigmax_1, sigmay_1 = 6.7, 6.7
theta0_1 = 0

initial_ax_1=(roi_1.mean(axis = 0)[-1] - roi_1.mean(axis = 0)[1])/len(roi_1.mean(axis = 0))
initial_by_1=(roi_1.mean(axis = 1)[-1] - roi_1.mean(axis = 1)[1])/len(roi_1.mean(axis = 1))
#             initial_guess_1 = (2,dif.allBragg_coord_2[i][0],dif.allBragg_coord_2[i][1],6.7,6.7,0, np.min(roi_1), initial_ax_1, initial_by_1)
initial_guess_1 = (amp0_1, x0_1, y0_1, sigmax_1, sigmay_1,theta0_1, offset0_1, initial_ax_1, initial_by_1)
ppot_tmp, pcov_tmp = opt.curve_fit(ued_dt.Gaussian_2D1_with_plane, xx_yy_1, roi_1.ravel(), p0=initial_guess_1)

axs = []
fig, axs = plt.subplots(ncols= 3, nrows = 2, figsize = (16/1.,8/1.))
# ppot_tmp .
# gauss_img = ued_dt.Gaussian_2D1_with_plane(xx_yy_1, *dif.bragg_fits_binned_delays[t_idx][i]).reshape(dif.roisize*2,dif.roisize*2)
# gauss_img = ued_dt.Gaussian_2D1_with_plane(xx_yy_1, *initial_guess_1).reshape(dif.roisize*2,dif.roisize*2)
gauss_img = ued_dt.Gaussian_2D1_with_plane(xx_yy_1, *ppot_tmp).reshape(dif.roisize*2,dif.roisize*2)
print('bragg fits 3:')
print(dif.static.bragg_fits3[0][i])
print('popt:')
print(ppot_tmp)

vmin = np.percentile(roi_1, 0)
vmax = np.percentile(roi_1, 100)

im0 = axs[0,0].pcolormesh(xx_yy_1[0], xx_yy_1[1],roi_1, vmin=vmin, vmax=vmax)
im1 = axs[0,1].pcolormesh(xx_yy_1[0], xx_yy_1[1], ued_dt.Gaussian_2D1_with_plane(xx_yy_1, *dif.bragg_fits_binned_delays[t_idx][i]).reshape(dif.roisize*2,dif.roisize*2), vmin=vmin, vmax=vmax )
# plt.colorbar(im0, ax=axs[1])


vmin = np.percentile(roi_1 - ued_dt.Gaussian_2D1_with_plane(xx_yy_1, *dif.bragg_fits_binned_delays[t_idx][i]).reshape(dif.roisize*2,dif.roisize*2), 0)
vmax = np.percentile(roi_1 - ued_dt.Gaussian_2D1_with_plane(xx_yy_1, *dif.bragg_fits_binned_delays[t_idx][i]).reshape(dif.roisize*2,dif.roisize*2), 100)
vmax = np.max((np.abs(vmin), vmax))

im2 = axs[0,2].pcolormesh(xx_yy_1[0], xx_yy_1[1],roi_1 - ued_dt.Gaussian_2D1_with_plane(xx_yy_1, *dif.bragg_fits_binned_delays[t_idx][i]).reshape(dif.roisize*2,dif.roisize*2),cmap='seismic', vmin=-vmax, vmax=vmax )

plt.colorbar(im0, ax=axs[0,0])
plt.colorbar(im1, ax=axs[0,1])
plt.colorbar(im2, ax=axs[0,2])


axs[0, 0].set_title(f'exp: {dif.allBragg_indices_2[i]} peak')
axs[0, 1].set_title(f'2D Gaussian Fit')
axs[0, 2].set_title(f'difference empty_img2')



vmin = np.percentile(dif.first_image, 0)
vmax = np.percentile(dif.first_image, 80)
rect = patch.Rectangle(roi_1_lower_left, (roi_1_upper_right-roi_1_lower_left)[0], (roi_1_upper_right-roi_1_lower_left)[1] ,
                      edgecolor = 'red',fill=False, lw=2 )
im4 = axs[1,2].pcolormesh(np.log(dif.first_image), )
axs[1,2].add_patch(rect)
plt.colorbar(im4, ax=axs[1,2])

axs[1, 0].plot(dif.xx[roi_1_selx,:][:,roi_1_sely][0], roi_1[38-1:38+1,:].mean(axis=0), label = 'img - bkg')
axs[1, 0].plot(dif.xx[roi_1_selx,:][:,roi_1_sely][0], gauss_img[38-1:38+1,:].mean(axis=0), '--', label = 'fit over x')
axs[1, 0].plot(dif.xx[roi_1_selx,:][:,roi_1_sely][0], roi_1[38-1:38+1,:].mean(axis=0) - gauss_img[38-1:38+1,:].mean(axis=0), c= 'r', label = 'difference') 
axs[1, 0].axhline(0, c= 'k', ls='--', alpha = 0.3)
axs[1, 0].legend(frameon = False)

axs[1, 1].plot(dif.xx[roi_1_selx,:][:,roi_1_sely][1], roi_1[:,40-1:40+1].mean(axis = 1), label = 'img - bkg (Y)')
axs[1, 1].plot(dif.xx[roi_1_selx,:][:,roi_1_sely][1], gauss_img[:,40-1:40+1].mean(axis = 1), '--', label = 'fit over y')
axs[1, 1].plot(dif.xx[roi_1_selx,:][:,roi_1_sely][1], roi_1[:,40-1:40+1].mean(axis=1) - gauss_img[:,40-1:40+1].mean(axis=1), c= 'r', label = 'difference') 
axs[1, 1].axhline(0, c= 'k', ls='--', alpha = 0.3)
axs[1, 1].legend(frameon = False)
axs[0,0].axvline(xx_yy_1[0][0,40])
axs[0,0].axhline(xx_yy_1[1][40,0])
plt.colorbar(im0, ax=axs[1,0])
plt.colorbar(im1, ax=axs[1,1])

In [ ]:
q_tmp_odd = np.sqrt(3**2 + 0)
q_tmp_even = np.sqrt(3**2 + 1)
#Fe
print('Fe')
print('odd:')
print(np.exp(-0.25*q_tmp_odd**2))
print('even')
print(np.exp(-0.25*q_tmp_even**2))

#Fe
print('Pt')
print('300:')
print(np.exp(-0.15*q_tmp_odd**2))
print('310')
print(np.exp(-0.15*q_tmp_even**2))


In [ ]:
#code from the first fit... 

i=3
avg_fit = (0.5*(dif.static.bragg_fits2[i, ::-1, :] + dif.static.bragg_fits2[i,:,:]))[dif.sel_bragg_in_im][:-1]
test_sel = np.where((avg_fit[:,2] != 0.)&(avg_fit[:,1] != 0.))


cent_test2 = np.zeros((int(dif.max_frame/dif.frame_step),2))
for i in np.arange(int(dif.max_frame/dif.frame_step)):
    
    avg_fit = (0.5*(dif.static.bragg_fits2[i, ::-1, :] + dif.static.bragg_fits2[i,:,:]))[dif.sel_bragg_in_im][:-1]
    test_sel = np.where((avg_fit[:,2] != 0.)&(avg_fit[:,1] != 0.))
    
    cent_test2[i] = avg_fit[test_sel, 1:3].mean(axis = 1)[0]


In [ ]:
dif.a_list =         np.zeros(dif.bragg_fits_binned_delays[:,dif.sel_bragg_in_im,1:3 ].shape[0:2])
dif.a_list_errors =  np.zeros(dif.bragg_fits_binned_delays[:,dif.sel_bragg_in_im,1:3 ].shape[0:2])

for j in np.arange(len(dif.a_list)):    
    for i in np.arange(len(dif.a_list[0,:])):
        dif.a_list[j][i] = dt.find_a(dif.bragg_fits_binned_delays[j,dif.sel_bragg_in_im,1:3 ][i], 
                            dif.centers_time[j],dif.allBragg_indices_2[dif.sel_bragg_in_im][i], coef=dif.coef )
#                             dif.centerpos,dif.allBragg_indices_2[dif.sel_bragg_in_im][i], coef=dif.coef )
        
        dif.a_list_errors[j][i] = dt.find_a_error(dif.bragg_fits_binned_delays[j,dif.sel_bragg_in_im,1:3 ][i], 
                                     dif.centers_time[j],dif.allBragg_indices_2[dif.sel_bragg_in_im][i], 
#                                      dif.centerpos,dif.allBragg_indices_2[dif.sel_bragg_in_im][i], 
                                     
                                     dif.centers_time_errors[j],
                                     dif.fit_erros_binned_delays_sigma[j,dif.sel_bragg_in_im,1:3 ][i], 
                                     coef=dif.coef )
# dif.a_list = a_list
# dif.a_list_errors = a_list_errors

In [ ]:
dif.q_list = np.zeros(dif.bragg_fits_binned_delays[:,:,1:3 ].shape[0:2])
dif.q_list_errors = np.zeros(dif.bragg_fits_binned_delays[:,:,1:3 ].shape[0:2])

for j in np.arange(len(dif.q_list)):    
    for i in np.arange(len(dif.q_list[0,:])):
        dif.q_list[j][i] = dt.find_q(dif.bragg_fits_binned_delays[j,:,1:3 ][i], 
                            dif.centers_time[j], coef=dif.coef )
#                             dif.centerpos, coef=coef )

        dif.q_list_errors[j][i] = dt.find_q_error(dif.bragg_fits_binned_delays[j,:,1:3 ][i], 
                                     dif.centers_time[j],
#                                      dif.centerpos,
                                     
                                     dif.centers_time_errors[j],
                                     dif.fit_erros_binned_delays_sigma[j,:,1:3 ][i], 
                                     coef=dif.coef )
# dif.q_list = q_list
# dif.q_list_errors = q_list_errors

In [ ]:
plt.figure()
plt.errorbar(dif.bincenters, dif.q_list[:, 59]/dif.q_list[:, 59][0], yerr = dt.norm_error(dif.q_list[:, 59], dif.q_list_errors[:,59]))
plt.errorbar(dif.bincenters, dif.a_list[:, 40], yerr = dt.norm_error(dif.a_list[:, 40], dif.a_list_errors[:,40]))
# plt.errorbar(dif.bincenters, dif.a_list[:, 40]/dif.a_list[:, 40][0], yerr = dt.norm_error(dif.a_list[:, 40], dif.a_list_errors[:,40]))

plt.xlim(-2,8)
plt.ylim(0.994, 1.001)

In [ ]:
plt.plot(dif.a_list[0,:])

In [ ]:
plt.figure()
plt.errorbar(dif.bincenters, dif.q_list[:, 59]/dif.q_list[:, 59][0], yerr = dt.norm_error(dif.q_list[:, 59], dif.q_list_errors[:,59]))
plt.errorbar(dif.bincenters, dif.a_list[:, 40]/dif.a_list[:, 40][0], yerr = dt.norm_error(dif.a_list[:, 40], dif.a_list_errors[:,40]))
# plt.errorbar(dif.bincenters, dif.a_list[:, 40]/dif.a_list[:, 40][0], yerr = dt.norm_error(dif.a_list[:, 40], dif.a_list_errors[:,40]))

plt.xlim(-2,8)
plt.ylim(0.994, 1.001)

In [ ]:
fig = plt.figure(figsize = (9,9))
cmap = plt.get_cmap('cool')
j=0
plt.errorbar(np.arange(len(dif.a_list[j])), dif.a_list[j], yerr=dif.a_list_errors[j],ls='--', capsize=3)
for i, label in enumerate(dif.allBragg_indices_2[dif.sel_bragg_in_im]):
    plt.text(i, dif.a_list[j][i]+0.003, label, fontsize=9, ha='center')

plt.xlabel('Bragg spot #')
plt.ylabel('a (A)')
plt.legend(ncol = 3, fontsize = 6 )

# fig.savefig('test_fig.eps')
# plt.xlim(0,10)
# plt.xlim(0,50)

In [ ]:
plt.figure(figsize = (9,6))
cmap = plt.get_cmap('cool')

for j in np.arange(len(dif.a_list)):
    color = cmap(float(j)/len(dif.a_list))
    plt.plot(2*np.pi/(dif.a_list[j]), 
             color = color,
             label = f'{np.round(dif.bincenters[j], 3)} ps')
plt.xlabel('Bragg spot #')
plt.ylabel('a (A)')
plt.legend(ncol = 3, fontsize = 8 )
# plt.xlim(0, 140)
# plt.xlim(30, 40)

In [ ]:
sorted_braggs = np.argsort(dif.a_list[0])

In [ ]:
plt.figure()
plt.errorbar(np.arange(len(dif.a_list[0][sorted_braggs])),dif.a_list[0][sorted_braggs],  dif.a_list_errors[0][sorted_braggs])


In [ ]:
vmin = np.percentile(dif.first_image, 5)
vmax = np.percentile(dif.first_image, 98)
cmap = plt.get_cmap('cool')


plt.figure()
plt.pcolormesh(dif.first_image,vmin=vmin, vmax=vmax)
for i, pos in enumerate(dif.allBragg_coord_2[dif.sel_bragg_in_im][sorted_braggs]):
    color = cmap(float(i)/len(dif.allBragg_coord_2[dif.sel_bragg_in_im][sorted_braggs]))
    plt.scatter(pos[0], pos[1], color=color, label = f'{i} st')

In [ ]:
plt.figure()
for j in np.arange(len(dif.a_list)):   
    color = cmap(float(j)/len(dif.a_list))
    plt.plot(dif.a_list[j]/dif.a_list[0], c= color)

In [ ]:
sel_pret0 = dif.bincenters < -0.5

In [ ]:
dif.a_list_norm_errors = np.zeros_like(dif.a_list)
for i in range(dif.a_list_norm_errors.shape[1]):
    dif.a_list_norm_errors[:,i] = dt.norm_error(dif.a_list[:,i], dif.a_list_errors[:,i])
dif.a_list_mean_norm_errors = np.sqrt( np.sum((dif.a_list_norm_errors/dif.a_list_norm_errors.shape[1])**2, axis =1) )

In [ ]:
plt.figure()
for i in np.arange(dif.a_list[0,:].shape[0]):

    pass

    plt.errorbar(
        dif.bincenters,
        dif.a_list[:,i]/dif.a_list[sel_pret0,i].mean(), yerr=dif.a_list_norm_errors[:,i], color='k', alpha=0.1,  label='')
    

plt.errorbar(dif.bincenters,np.mean((dif.a_list[:,:]/dif.a_list[sel_pret0,:].mean(axis = 0)), axis=1),yerr=dif.a_list_mean_norm_errors, ls='-',marker ='.', label='', color = 'red',zorder=np.inf)

plt.xlabel('Delay (ps)')
plt.ylabel('$a/a_0$')
plt.xlim(-2.5,20)


In [ ]:
igor_bragg_list = [
    [1,0,0],
#     [1,0,0],
#     [1,1,0],
    [2,0,0],
#     [2,1,0],
#     [2,2,0],
    [3,0,0],
    [4,0,0],
    [5,0,0],
#     [6,0,0],
#     [5,1,0]
#     [2,2,0],
#     [3,0,0],
#     [3,1,0],
#     [3,2,0],
#     [4,0,0],
#     [4,1,0],
#     [3,3,0]
]

## Position and Amplitude of different Braggs over time

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows = 2, figsize = (10, 20))
# fig, [ax1, ax2] = plt.subplots(nrows = 2, figsize = (5/1.2/3, 10.2/1.2))

cmap = plt.get_cmap('rainbow')
for j, bragg in enumerate(igor_bragg_list):
    sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
    fits = dif.bragg_fits_binned_delays[:,sel_bragg,:]    
    fit_errors = dif.fit_erros_binned_delays_sigma[:,sel_bragg,:] 
    
    q = dif.q_list[:,sel_bragg][:,0]
    q_error = dif.q_list_errors[:,sel_bragg][:,0]
    
    I_tmp = fits[:,0,0]
    I_tmp_error = fit_errors[:,0,0]
    
    I_tmp_norm = I_tmp/I_tmp[0]
    I_tmp_norm_error = dt.norm_error(I_tmp, I_tmp_error) 
    
    q_norm = q/q[0]
    q_norm_error = dt.norm_error(q, q_error) 

    color = cmap((len(igor_bragg_list) - float(j))/len(igor_bragg_list))
    
    ax1.errorbar(dif.bincenters, I_tmp_norm,yerr=I_tmp_norm_error,ls='-', color = color, label = bragg)    
    ax2.errorbar(dif.bincenters, q_norm, yerr=1*q_norm_error, ls='-',  color = color, label = bragg)
    
ax1.set_xlabel('Delay (ps)', fontsize = 12)
ax1.set_ylabel('$A/A_0$', fontsize = 12)
# ax1.legend(fontsize = 12)
ax1.set_xlim(-2,8)
# ax1.set_xlim(8, 100)



ax2.set_xlabel('Delay (ps)', fontsize = 12)
ax2.set_ylabel('$q/q_0$', fontsize = 12)
ax2.legend(ncol = 2, fontsize = 9.5)
ax1.axhline(1, c='k')
ax2.set_xlim(-2,8)
# ax2.set_xlim(8, 100)

plt.tight_layout()

# fig.savefig('paper_figs_tmp/peak_dynamics', dpi = 300)

#### Other parameters of the Fit 

In [ ]:
fig, [[ax1, ax2], [ax3, ax4], [ax5, ax6], [ax7, ax8]] = plt.subplots(ncols = 2,nrows = 4, figsize = (10/1.0,12/1.0))
cmap = plt.get_cmap('rainbow')
for j, bragg in enumerate(igor_bragg_list):
    sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
    
    fits = dif.bragg_fits_binned_delays[:,sel_bragg,:]    
    fit_errors = dif.fit_erros_binned_delays_sigma[:,sel_bragg,:]  
    
    q = dif.q_list[:,sel_bragg][:,0]
    q_error = dif.q_list_errors[:,sel_bragg][:,0]
    
    I_tmp = fits[:,0,0]
    I_tmp_error = fit_errors[:,0,0]
    
    I_tmp_norm = I_tmp/I_tmp[0]
    I_tmp_norm_error = dt.norm_error(I_tmp, I_tmp_error) 
    
    q_norm = q/q[0]
    q_norm_error = dt.norm_error(q, q_error) 
    
    
    
#     fits = bragg_fits_binned_delays[:,sel_bragg,:]
# #     q = np.sqrt((fits[:,0,1] - dif.centerpos[0])**2 + (fits[:,0,2] - dif.centerpos[1])**2)
#     q = np.sqrt((fits[:,0,1] - dif.centerpos_time[:,0])**2 + (fits[:,0,2] - dif.centerpos_time[:,1])**2)
    
    color = cmap((len(igor_bragg_list) - float(j))/len(igor_bragg_list))
    
    
    ax2.plot(dif.bincenters, fits[:,0,0]-fits[:,0,0][0], '.-', color = color, label = bragg)
#     ax1.plot(dif.bincenters, fits[:,0,0], '.-', color = color, label = bragg)

    ax3.plot(dif.bincenters, np.abs(fits[:,0,3])/np.abs(fits[:,0,3][0]), '.-', color = color, label = bragg)
    ax4.plot(dif.bincenters, np.abs(fits[:,0,4])/np.abs(fits[:,0,4][0]), '.-', color = color, label = bragg)
#     ax4.plot(dif.bincenters, ((fits[:,0,4]*fits[:,0,3])-(fits[:,0,4][0]*fits[:,0,3][0]))/((fits[:,0,4]*fits[:,0,3])-(fits[:,0,4][0]*fits[:,0,3][0]))[-1], '.-', color = color, label = bragg)
    ax7.plot(dif.bincenters, (fits[:,0,3]), '.-', color = color, label = bragg)
    ax7.plot(dif.bincenters, (fits[:,0,4]), '--', color = color, label = bragg)
    ax8.plot(dif.bincenters, (fits[:,0,3]*fits[:,0,4])/((fits[:,0,3]*fits[:,0,4])[0]), '--', color = color, label = bragg)
    
    ax5.plot(dif.bincenters, np.mod(fits[:,0,5], 2*np.pi)/np.mod(fits[:,0,5][0], 2*np.pi), '.-', color = color, label = bragg)
#     ax5.plot(dif.bincenters, fits[:,0,5]-fits[:,0,5][0], '.-', color = color, label = bragg)

    ax6.plot(dif.bincenters, (fits[:,0,6])/(fits[:,0,6])[-1], '.-', color = color, label = bragg)
#     ax6.plot(dif.bincenters, fits[:,0,6]-fits[:,0,6][0], '.-', color = color, label = bragg)


#     ax1.plot(dif.bincenters, q_norm, '.-', color = color, label = bragg)
    ax1.errorbar(dif.bincenters, q_norm, yerr=q_norm_error, color = color, label = bragg)
    
ax2.set_xlabel('Delay (ps)')
ax2.set_ylabel('$A/A_0$')
ax2.legend(fontsize = 6)
ax2.set_xlim(-2,8)
ax2.set_ylim(-0.5,1)


ax1.set_xlabel('Delay (ps)')
ax1.set_ylabel('$q/q_0$')
ax1.legend(fontsize = 6)
ax1.set_xlim(-2,8)



ax3.set_xlabel('Delay (ps)')
ax3.set_ylabel('$\sigma_x / \sigma_{x0}$')
ax3.legend(fontsize = 6)
ax3.set_xlim(-2,8)

ax4.set_xlabel('Delay (ps)')
ax4.set_ylabel('$\sigma_y / \sigma_{y0}$')
ax4.legend(fontsize = 6)
ax4.set_xlim(-2,8)
# ax4.set_ylim(0.98,1.02)


ax5.set_xlabel('Delay (ps)')
ax5.set_ylabel('$ \Theta / \Theta_0 $')
ax5.legend(fontsize = 6)
ax5.set_xlim(-2,8)
# ax5.set_ylim(-1/5,1/5)
ax5.set_ylim(-2.5,2.5)

ax6.set_xlabel('Delay (ps)')
ax6.set_ylabel('$Offset / Offset_0$')
ax6.legend(fontsize = 6)
# ax6.set_ylim(0.75,2)

ax6.set_xlim(-2,8)

ax7.set_xlim(-2,8)
ax8.set_xlim(-2,8)


In [ ]:
igor_bragg_list_tmp = [
#     [1,0,0],
#     [-1,0,0],
#     [0,1,0],
#     [0,-1,0],
    
    [2,1,0],
    [-2,-1,0],
    [-1,2,0],
    [1,-2,0],
    
]

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))
for j, bragg in enumerate(igor_bragg_list_tmp):
    sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])

    
     
    
    
    
    fits = dif.bragg_fits_binned_delays[:,sel_bragg,:]    
    fit_errors = dif.fit_erros_binned_delays_sigma[:,sel_bragg,:]  
    
    I_tmp = fits[:,0,0]*fits[:,0,4]*fits[:,0,3]
    I_tmp_error = fit_errors[:,0,0]*fits[:,0,4]*fits[:,0,3]
    
    I_tmp_norm = I_tmp/I_tmp[0]
    I_tmp_norm_error = dt.norm_error(I_tmp, I_tmp_error) 
    
    bragg_pos_tmp = fits[:,0,1:3]
    bragg_pos_error_tmp = dif.fit_erros_binned_delays_sigma[:,0,1:3]
    
    I_vol_tmp = fits[:,0,0]*fits[:,0,4]*fits[:,0,3]
    I_vol_err_tmp = (I_vol_tmp)*np.sqrt( (fit_errors[:,0,0]/fits[:,0,0])**2 + (fit_errors[:,0,3]/fits[:,0,3])**2 + (fit_errors[:,0,4]/fits[:,0,4])**2 )
    
    
#     q = find_q(bragg_pos_tmp.T, dif.centerpos_time.T, coef=coef)
#     q_error = find_q_error(bragg_pos_tmp.T, dif.centerpos_time.T, dif.centers_time_errors.T, bragg_pos_error_tmp.T, coef = coef)
    
#     q_norm = q/q[0]
#     q_norm_error = norm_error(q, q_error) 
    
    color = cmap((len(igor_bragg_list) - float(j))/len(igor_bragg_list))
    ax.errorbar(dif.bincenters, I_vol_tmp/I_vol_tmp[0], yerr=dt.norm_error(I_vol_tmp, I_vol_err_tmp) ,marker='.',  color = color, label = bragg, alpha = 0.5)
    

for j, bragg in enumerate(igor_bragg_list_tmp):
    try:
        sel_bragg_sym = ((dif.compact_hkl[:,0] == np.abs(bragg[0])))&(dif.compact_hkl[:,1] == np.abs(bragg[1]))#or(dif.compact_hkl[:,0] == np.abs(bragg[1]))&(dif.compact_hkl[:,1] == np.abs(bragg[0]))    
        amp_corrected = dif.volumes_symmetrised_compact[np.where(sel_bragg_sym)[0][0]]
        I_tmp_sym = amp_corrected
        I_tmp_error_sym =  dif.volumes_symmetrised_compact_errors[np.where(sel_bragg_sym)[0][0]]
        ax.plot(dif.bincenters, I_tmp_sym/I_tmp_sym[0], c= 'k' )
    except:
        pass
        


ax.set_xlabel('Delay (ps)')
ax.set_ylabel('$ (\sigma_x x \sigma_y / ( \sigma_{x0} x \sigma_{y0} )$')
ax.legend(fontsize = 6)
ax.set_xlim(-2,8)
ax.set_title('Area of Bragg')




## Create a symetrised dataset for better SNR by exploiting symmetry

In [ ]:
# create vectors where to put the info


dif.bragg_fits_simetrised = np.zeros_like(dif.bragg_fits_binned_delays)## all values are legit exept the position that will probably avg to zero ... 
                                                               #should make something for that at some point I guess
    
dif.bragg_fits_simetrised_q = np.zeros((dif.bragg_fits_binned_delays.shape[0],dif.bragg_fits_binned_delays.shape[1]) )
dif.bragg_fits_simetrised_areas = np.zeros((dif.bragg_fits_binned_delays.shape[0],dif.bragg_fits_binned_delays.shape[1]) )
dif.bragg_fits_simetrised_volumes = np.zeros((dif.bragg_fits_binned_delays.shape[0],dif.bragg_fits_binned_delays.shape[1]) )

dif.nb_of_eqivalent_braggs = np.zeros(len(dif.allBragg_indices_2))

# q_vecs = copy.copy(dif.q_list) #np.zeros((len(dif.bincenters), bragg_fits_binned_delays.shape[1]))
bragg_areas = np.zeros((len(dif.bincenters), dif.bragg_fits_binned_delays.shape[1]))
bragg_volumes = np.zeros((len(dif.bincenters), dif.bragg_fits_binned_delays.shape[1]))

dif.bragg_fits_simetrised_errors = np.zeros_like(dif.bragg_fits_simetrised )

dif.bragg_fits_simetrised_q_errors = np.zeros_like(dif.bragg_fits_simetrised_q )
dif.bragg_fits_simetrised_areas_errors = np.zeros_like(dif.bragg_fits_simetrised_areas )
dif.bragg_fits_simetrised_volumes_errors = np.zeros_like(dif.bragg_fits_simetrised_volumes )

dif.nb_of_eqivalent_braggs_errors = np.zeros_like(dif.nb_of_eqivalent_braggs )

# q_vecs_errors = copy.copy(dif.q_list_errors)
bragg_areas_errors = np.zeros_like(bragg_areas)
bragg_volumes_errors = np.zeros_like(bragg_volumes)

#this is a first loop where I only need to loop over time
# with this I can write bargg areas and Bragg erros... turned out not to be so necessary in the end 
for t_idx in range(len(dif.bincenters)):
    print(t_idx)
#     q_vecs[t_idx] = coef*np.sqrt((bragg_fits_binned_delays[t_idx,:,1] - dif.centerpos_time[t_idx,0])**2 + (bragg_fits_binned_delays[t_idx,:,2] - dif.centerpos_time[t_idx,1])**2)
    bragg_areas[t_idx] = dif.bragg_fits_binned_delays[t_idx,:,3]*dif.bragg_fits_binned_delays[t_idx,:,4]
#     print(dif.bragg_fits_binned_delays[t_idx,:,3])
    bragg_areas_errors[t_idx] = np.sqrt( bragg_areas[t_idx]*( (dif.fit_erros_binned_delays_sigma[t_idx,:,3]/dif.bragg_fits_binned_delays[t_idx,:,3])**2 +
                                                             (dif.fit_erros_binned_delays_sigma[t_idx,:,4]/dif.bragg_fits_binned_delays[t_idx,:,4])**2 ) )
#     print(bragg_areas_errors[t_idx])
    
    bragg_volumes[t_idx]= dif.bragg_fits_binned_delays[t_idx,:,0]*dif.bragg_fits_binned_delays[t_idx,:,3]*dif.bragg_fits_binned_delays[t_idx,:,4]
    bragg_volumes_errors[t_idx] = np.sqrt( bragg_volumes[t_idx]*( (dif.fit_erros_binned_delays_sigma[t_idx,:,0]/dif.bragg_fits_binned_delays[t_idx,:,0])**2 +
                                                                 (dif.fit_erros_binned_delays_sigma[t_idx,:,3]/dif.bragg_fits_binned_delays[t_idx,:,3])**2 +
                                                             (dif.fit_erros_binned_delays_sigma[t_idx,:,4]/dif.bragg_fits_binned_delays[t_idx,:,4])**2 ) )
####
###Here is where I save symmetrised peaks, the information here is still degenerated, but it's useful to work with older functions
for i in range(120):
    h,k,l = dif.allBragg_indices_2[i]
    idx_list = I_UED_dt.select_braggs_Bragg(h,k, dif.allBragg_indices_2)
    dif.nb_of_eqivalent_braggs[i] = np.unique(idx_list).shape[0]
    
    dif.bragg_fits_simetrised[:,i] = dif.bragg_fits_binned_delays[:,idx_list].mean(axis = 1)
    dif.bragg_fits_simetrised_q[:,i] = dif.q_list[:, idx_list].mean(axis = 1)   
    dif.bragg_fits_simetrised_areas[:,i] = bragg_areas[:, idx_list].mean(axis = 1)   
    dif.bragg_fits_simetrised_volumes[:,i] = bragg_volumes[:, idx_list].mean(axis = 1)   
    
    dif.bragg_fits_simetrised_errors[:,i] = dif.fit_erros_binned_delays_sigma[:,idx_list].mean(axis = 1)/np.sqrt(dif.nb_of_eqivalent_braggs[i])
    dif.bragg_fits_simetrised_q_errors[:,i] = dif.q_list_errors[:, idx_list].mean(axis = 1) /np.sqrt(dif.nb_of_eqivalent_braggs[i])
    dif.bragg_fits_simetrised_areas_errors[:,i] = bragg_areas_errors[:, idx_list].mean(axis = 1) /np.sqrt(dif.nb_of_eqivalent_braggs[i])   
    dif.bragg_fits_simetrised_volumes_errors[:,i] = bragg_volumes_errors[:, idx_list].mean(axis = 1) /np.sqrt(dif.nb_of_eqivalent_braggs[i])   


# # Here is where I save symmetrised peaks, the information here is still degenerated, but it's useful to work with older functions
# for i in range(120):
# #     print(i)
#     h,k,l = dif.allBragg_indices_2[i]
#     idx_list = I_UED_dt.select_braggs_Bragg(h,k, dif.allBragg_indices_2)
#     dif.nb_of_eqivalent_braggs[i] = np.unique(idx_list).shape[0]
# #     print( np.unique(idx_list).shape[0])
    
#     dif.bragg_fits_simetrised[:,i] = np.nanmean(dif.bragg_fits_binned_delays[:,idx_list], axis = 1)
#     dif.bragg_fits_simetrised_q[:,i] = np.nanmean(dif.q_list[:, idx_list], axis = 1)   
#     dif.bragg_fits_simetrised_areas[:,i] = np.nanmean(bragg_areas[:, idx_list], axis = 1)   
#     dif.bragg_fits_simetrised_volumes[:,i] = np.nanmean(bragg_volumes[:, idx_list], axis = 1)   
    
#     dif.bragg_fits_simetrised_errors[:,i] = np.nanmean(dif.fit_erros_binned_delays_sigma[:,idx_list], axis=1)/np.sqrt(dif.nb_of_eqivalent_braggs[i])
#     dif.bragg_fits_simetrised_q_errors[:,i] = np.nanmean(dif.q_list_errors[:, idx_list], axis=1) /np.sqrt(dif.nb_of_eqivalent_braggs[i])
#     dif.bragg_fits_simetrised_areas_errors[:,i] = np.nanmean(bragg_areas_errors[:, idx_list], axis=1) /np.sqrt(dif.nb_of_eqivalent_braggs[i])   
#     dif.bragg_fits_simetrised_volumes_error"s[:,i] = np.nanmean(bragg_volumes_errors[:, idx_list], axis=1) /np.sqrt(dif.nb_of_eqivalent_braggs[i])   


# # Save info to dif container
# dif.bragg_fits_simetrised  = bragg_fits_simetrised
# dif.bragg_fits_simetrised_q = bragg_fits_simetrised_q
# dif.bragg_fits_simetrised_areas = bragg_fits_simetrised_areas
# dif.bragg_fits_simetrised_errors = bragg_fits_simetrised_errors
# dif.bragg_fits_simetrised_q_errors = bragg_fits_simetrised_q_errors
# dif.bragg_fits_simetrised_areas_errors = bragg_fits_simetrised_areas_errors

### Here I create a compact array, where I only keep 1 Bragg spot value from all of the symmetric ones 

In [ ]:
np.sum(~np.isnan(dif.bragg_fits_simetrised_volumes_errors))

In [ ]:
tmp_idx_arr = np.arange(120)
tmp_idx_arr2 = np.arange(120)

already_done = np.array([])
dif.compact = np.array([ dif.bragg_fits_simetrised[:,0,:] ])
dif.compact_errors = np.array([ dif.bragg_fits_simetrised[:,0,:] ])

dif.q_symmetrised_compact = np.array([ dif.bragg_fits_simetrised_q[:,0] ])
dif.q_symmetrised_compact_errors = np.array([ dif.bragg_fits_simetrised_q[:,0] ])

dif.areas_symmetrised_compact = np.array([ dif.bragg_fits_simetrised_q[:,0] ])
dif.areas_symmetrised_compact_errors = np.array([ dif.bragg_fits_simetrised_q[:,0] ])

dif.volumes_symmetrised_compact = np.array([ dif.bragg_fits_simetrised_q[:,0] ])
dif.volumes_symmetrised_compact_errors = np.array([ dif.bragg_fits_simetrised_q[:,0] ])

dif.compact_hkl = np.array([(0,0)])

count = 0 

for i in tmp_idx_arr:
    
    h,k,l = dif.allBragg_indices_2[i]
    idxs = np.unique(I_UED_dt.select_braggs_Bragg(h,k, dif.allBragg_indices_2) )
    
    if (idxs[0] not in already_done):
        already_done = np.append(already_done, idxs)
        dif.compact_hkl =  np.append(dif.compact_hkl, [[h,k]],axis = 0 )
        
        dif.q_symmetrised_compact = np.append(dif.q_symmetrised_compact, [dif.bragg_fits_simetrised_q[:,i]], axis = 0)
        dif.q_symmetrised_compact_errors = np.append(dif.q_symmetrised_compact_errors, [dif.bragg_fits_simetrised_q_errors[:,i]], axis = 0)
        
        dif.areas_symmetrised_compact = np.append(dif.areas_symmetrised_compact, [dif.bragg_fits_simetrised_areas[:,i]], axis = 0)
        dif.areas_symmetrised_compact_errors = np.append(dif.areas_symmetrised_compact_errors, [dif.bragg_fits_simetrised_areas_errors[:,i]], axis = 0)
        
        dif.volumes_symmetrised_compact = np.append(dif.volumes_symmetrised_compact, [dif.bragg_fits_simetrised_volumes[:,i]], axis = 0)
        dif.volumes_symmetrised_compact_errors = np.append(dif.volumes_symmetrised_compact_errors, [dif.bragg_fits_simetrised_volumes_errors[:,i]], axis = 0)
        
        dif.compact = np.append(dif.compact, [dif.bragg_fits_simetrised[:,i,:]], axis = 0 )
        dif.compact_errors = np.append(dif.compact_errors, [dif.bragg_fits_simetrised_errors[:,i,:]], axis = 0 )
        count += 1
        
        
dif.compact_hkl= dif.compact_hkl[1:]*(-1)  

dif.q_symmetrised_compact = dif.q_symmetrised_compact[1:]
dif.q_symmetrised_compact_errors = dif.q_symmetrised_compact_errors[1:]

dif.areas_symmetrised_compact = dif.areas_symmetrised_compact[1:]
dif.areas_symmetrised_compact_errors = dif.areas_symmetrised_compact_errors[1:]

dif.volumes_symmetrised_compact = dif.volumes_symmetrised_compact[1:]
dif.volumes_symmetrised_compact_errors = dif.volumes_symmetrised_compact_errors[1:]

dif.compact = dif.compact[1:]
dif.compact_errors = dif.compact_errors[1:]

# And now we save into the dif container again


# dif.compact_hkl = compact_hkl
# dif.q_symmetrised_compact = q_symmetrised_compact
# dif.q_symmetrised_compact_errors = q_symmetrised_compact_errors
# dif.areas_symmetrised_compact = areas_symmetrised_compact
# dif.areas_symmetrised_compact_errors = areas_symmetrised_compact_errors
# dif.compact = compact
# dif.compact_errors  = compact_errors



In [ ]:
As = dif.compact[:,:,0]*dif.compact[:,:,3]*dif.compact[:,:,4]
As2 = dif.compact[:,:,0]*dif.areas_symmetrised_compact

As_errors = np.abs(As)*np.sqrt( (dif.compact_errors[:,:,0]/dif.compact[:,:,0])**2 + (dif.compact_errors[:,:,3]/dif.compact[:,:,3])**2 + (dif.compact_errors[:,:,4]/dif.compact[:,:,4])**2 )
As2_errors = np.abs(As2)*np.sqrt( (dif.compact_errors[:,:,0]/dif.compact[:,:,0])**2 + (dif.areas_symmetrised_compact_errors/dif.areas_symmetrised_compact)**2 )

In [ ]:
igor_bragg_list_even = [
#    [1, 0],
   [1, 1],
   [2, 0],
#    [2, 1],
   [2, 2],
#    [3, 0],
   [3, 1],
#    [3, 2],
   [3, 3],
   [4, 0],
#    [4, 1],
   [4, 2],
#    [4, 3],
#    [4, 4], ## bad fit
#    [5, 0],
#    [5, 1], ## bad fit
#    [5, 2],
#    [5, 3], ##bad fit
#    [5, 4],
#    [5, 5]## bad fit
]

In [ ]:
igor_bragg_list_odd = [
   [1, 0],
#    [1, 1],
   [2, 0],
#    [2, 1],
#    [2, 2],
   [3, 0],
#    [3, 1],
#    [3, 2],
#    [3, 3],
   [4, 0],
#    [4, 1],
#    [4, 2],
#    [4, 3],
#    [4, 4], ## bad fit
#    [5, 0],
#    [5, 1], ## bad fit
#    [5, 2],
#    [5, 3], ##bad fit
#    [5, 4],
#    [5, 5]## bad fit
]

In [ ]:
plt.figure(figsize = (8,8))
# plt.errorbar(dif.bincenters, As[19]/As[19,0],yerr = dt.norm_error(As[19], As_errors[19]), marker ='.', c='C0', label = 'Area')
# plt.errorbar(dif.bincenters, As[18]/As[18,0],yerr = dt.norm_error(As[18], As_errors[18]), marker ='.', c='C1', label = 'Area')
# plt.errorbar(dif.bincenters, As[17]/As[17,0],yerr = dt.norm_error(As[17], As_errors[17]), marker ='.', c='C2', label = 'Area')
# plt.errorbar(dif.bincenters, As[16]/As[16,0],yerr = dt.norm_error(As[16], As_errors[16]), marker ='.', c='C3', label = 'Area')
# plt.errorbar(dif.bincenters, As[15]/As[15,0],yerr = dt.norm_error(As[15], As_errors[15]), marker ='.', c='C4', label = 'Area')


# for j, bragg in enumerate(igor_bragg_list_even):
#     sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
#     sel_bragg_sym = (dif.compact_hkl[:,0] == bragg[0])&(dif.compact_hkl[:,1] == bragg[1])
    
#     fits = dif.compact[sel_bragg_sym]
#     fit_errors = dif.compact_errors[sel_bragg_sym]
#     qs = dif.q_symmetrised_compact[sel_bragg_sym]
    
#     fits = np.swapaxes(fits,0,1)
#     fit_errors = np.swapaxes(fit_errors,0,1)
#     qs = np.swapaxes(qs,0,1)
    
#     fits = dif.bragg_fits_binned_delays[:,sel_bragg,:]    
#     fit_errors = dif.fit_erros_binned_delays_sigma[:,sel_bragg,:]  
    
#     I_vol = dif.bragg_fits_simetrised_volumes[:,sel_bragg][:,0]
#     I_vol_error = dif.bragg_fits_simetrised_volumes_errors[:,sel_bragg][:,0]
    
#     I_tmp = fits[:,0,0]
#     I_tmp_error = fit_errors[:,0,0]
    
#     I_tmp_norm = I_tmp/I_tmp[0]
#     I_tmp_norm_error = dt.norm_error(I_tmp, I_tmp_error) 
    
#     bragg_pos_tmp = fits[:,0,1:3]
#     bragg_pos_error_tmp = dif.fit_erros_binned_delays_sigma[:,0,1:3]
    
#     I_vol_tmp2 = fits[:,0,0]*fits[:,0,4]*fits[:,0,3]
#     I_vol_err_tmp2 = np.abs(I_vol_tmp2)*np.sqrt( (fit_errors[:,0,0]/fits[:,0,0])**2 + (fit_errors[:,0,3]/fits[:,0,3])**2 + (fit_errors[:,0,4]/fits[:,0,4])**2 )
    
# #     color = f'C{j}'
#     color = cmap((len(igor_bragg_list_even) - float(j))/len(igor_bragg_list_even)*3)
# #     plt.errorbar(dif.bincenters, I_tmp/I_tmp[0], yerr=dt.norm_error(I_tmp, I_tmp_error) ,ls='-.',  color = color, label = str(bragg) + ': A/A0' )
# #     plt.errorbar(dif.bincenters, I_vol/I_vol[0], yerr=dt.norm_error(I_vol, I_vol_error) ,ls='-',  color = color, label = str(bragg) + ': V/V0' )
    
#     plt.plot(dif.bincenters, qs/qs[0:5].mean(), color=color, label = bragg)

for j, bragg in enumerate(igor_bragg_list_odd):
    sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
    sel_bragg_sym = (dif.compact_hkl[:,0] == bragg[0])&(dif.compact_hkl[:,1] == bragg[1])
    
    fits = dif.compact[sel_bragg_sym]
    fit_errors = dif.compact_errors[sel_bragg_sym]
    qs = dif.q_symmetrised_compact[sel_bragg_sym]
    
    fits = np.swapaxes(fits,0,1)
    fit_errors = np.swapaxes(fit_errors,0,1)
    qs = np.swapaxes(qs,0,1)
    
    fits = dif.bragg_fits_binned_delays[:,sel_bragg,:]    
    fit_errors = dif.fit_erros_binned_delays_sigma[:,sel_bragg,:]  
    
    I_vol = dif.bragg_fits_simetrised_volumes[:,sel_bragg][:,0]
    I_vol_error = dif.bragg_fits_simetrised_volumes_errors[:,sel_bragg][:,0]
    
    I_tmp = fits[:,0,0]
    I_tmp_error = fit_errors[:,0,0]
    
    I_tmp_norm = I_tmp/I_tmp[0]
    I_tmp_norm_error = dt.norm_error(I_tmp, I_tmp_error) 
    
    bragg_pos_tmp = fits[:,0,1:3]
    bragg_pos_error_tmp = dif.fit_erros_binned_delays_sigma[:,0,1:3]
    
    I_vol_tmp2 = fits[:,0,0]*fits[:,0,4]*fits[:,0,3]
    I_vol_err_tmp2 = np.abs(I_vol_tmp2)*np.sqrt( (fit_errors[:,0,0]/fits[:,0,0])**2 + (fit_errors[:,0,3]/fits[:,0,3])**2 + (fit_errors[:,0,4]/fits[:,0,4])**2 )
    
#     color = f'C{j}'
    color = cmap((float(j))/len(igor_bragg_list_odd)*3)
#     plt.errorbar(dif.bincenters, I_tmp/I_tmp[0], yerr=dt.norm_error(I_tmp, I_tmp_error) ,ls='-.',  color = color, label = str(bragg) + ': A/A0' )
#     plt.errorbar(dif.bincenters, I_vol/I_vol[0], yerr=dt.norm_error(I_vol, I_vol_error) ,ls='-',  color = color, label = str(bragg) + ': V/V0' )
    
    plt.plot(dif.bincenters, qs/qs[0:5].mean(), color=color, label = bragg)



plt.xlim(-2,8)
plt.title('Comparison between Bragg peak area evolution \n  over time vs Bragg peak Amplitude')
plt.legend(frameon = False)

In [ ]:
igor_bragg_list = [
   [1,0,0],
   [1,1,0],
   [2,0,0],
   [2,1,0],
   [2,2,0],
   [3,0,0],
    [3,1,0],
    [3,2,0],
    [4,0,0],
    [4,1,0],
#     [4,2,0], #over 10A-1
    [3,3,0]
]

In [ ]:
igor_bragg_list_even = [
#    [1,0,0],
   [1,1,0],
   [2,0,0],
#    [2,1,0],
   [2,2,0],
#    [3,0,0],
    [3,1,0],
#     [3,2,0],
    [4,0,0],
#     [4,1,0],
#     [4,2,0], #over 10A-1
    [3,3,0]
]

In [ ]:
q_even_list = np.zeros((len(igor_bragg_list_even), len(dif.bincenters)))
q_avg_even = np.zeros(len(dif.bincenters))

for j, bragg in enumerate(igor_bragg_list_even):
#     sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
    sel_bragg_sym = (dif.compact_hkl[:,0] == bragg[0])&(dif.compact_hkl[:,1] == bragg[1])
    q = dif.q_symmetrised_compact[sel_bragg_sym][0]
    q_avg_even += q/q[0:5].mean()
    q_even_list[j] = q/q[0:5].mean()
    
q_avg_even = q_avg_even/len(igor_bragg_list_even)

In [ ]:
igor_bragg_list_odd = [
   [1,0,0],
#    [1,1,0],
#    [2,0,0],
   [2,1,0],
#    [2,2,0],
   [3,0,0],
#     [3,1,0],
    [3,2,0],
#     [4,0,0],
    [4,1,0],
#     [4,2,0], #over 10A-1
#     [3,3,0]
]

In [ ]:
q_odd_list = np.zeros((len(igor_bragg_list_odd), len(dif.bincenters)))
q_avg_odd = np.zeros(len(dif.bincenters))
for j, bragg in enumerate(igor_bragg_list_odd):
#     sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
    sel_bragg_sym = (dif.compact_hkl[:,0] == bragg[0])&(dif.compact_hkl[:,1] == bragg[1])
    q = dif.q_symmetrised_compact[sel_bragg_sym][0]
    q_avg_odd += q/q[0:5].mean()
    q_odd_list[j] = q/q[0:5].mean()
    
    
q_avg_odd = q_avg_odd/len(igor_bragg_list_odd)

In [ ]:
igor_bragg_list = [
   [1,0,0],
   [1,1,0],
   [2,0,0],
   [2,1,0],
   [2,2,0],
   [3,0,0],
    [3,1,0],
    [3,2,0],
    [4,0,0],
    [4,1,0],
#     [4,2,0], #over 10A-1
    [3,3,0]
]

In [ ]:
fig, [ax1,ax2] = plt.subplots(ncols=2,nrows=1, gridspec_kw={'width_ratios': [3, 1]}, figsize = (8,6),dpi=300)# sharex=True)

fig.subplots_adjust(hspace=.05)
fig.subplots_adjust(wspace=.05)
# fig.tight_layout()

ax1.axhline(1, lw=0.5, c='k')
ax2.axhline(1, lw=0.5, c='k')
cmap = plt.get_cmap('rainbow')
for j, bragg in enumerate(igor_bragg_list):
    
#     sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
    sel_bragg_sym = (dif.compact_hkl[:,0] == bragg[0])&(dif.compact_hkl[:,1] == bragg[1])
#     print(sel_bragg_sym)
#     fits = dif.compact[sel_bragg_sym]
#     fit_errors = dif.compact_errors[sel_bragg_sym]
#     fits = np.swapaxes(fits,0,1)
#     fit_errors = np.swapaxes(fit_errors,0,1)
    q = dif.q_symmetrised_compact[sel_bragg_sym][0]
    q_error = dif.q_symmetrised_compact_errors[sel_bragg_sym][0]

#     amp_corrected = dif.volumes_symmetrised_compact[np.where(sel_bragg_sym)[0][0]]

#     I_tmp = amp_corrected
#     I_tmp_error = fit_errors[:,0,0]
    
#     color = cmap((len(igor_bragg_list) - float(j+1))/(len(igor_bragg_list)+1))
    color = cmap((len(igor_bragg_list) - float(j))/(len(igor_bragg_list)))

#     ax1.errorbar(dif.bincenters, q/q[0:5].mean(),yerr = dt.norm_error(q, q_error), ls='-',marker='',zorder=-1, color = color, alpha = 0.5, label = bragg)
#     ax2.errorbar(dif.bincenters, q/q[0:5].mean(),yerr = dt.norm_error(q, q_error), ls='-',marker='',zorder=-1, color = color, alpha = 0.5, label = bragg)
    ax1.errorbar(dif.bincenters, q/q[0:5].mean(),yerr = dt.norm_error(q, q_error), ls='-',marker='',zorder=-1, color = color, alpha = 1, label = bragg)
    ax2.errorbar(dif.bincenters, q/q[0:5].mean(),yerr = dt.norm_error(q, q_error), ls='-',marker='',zorder=-1, color = color, alpha = 1, label = bragg)

# ax1.errorbar(dif.bincenters, q_avg_even, yerr=np.std(q_even_list, axis=0), ls='-', c='r', zorder=1, lw=2, label = 'average even peaks')
# ax1.errorbar(dif.bincenters, q_avg_odd, yerr=np.std(q_odd_list, axis = 0), ls='-', c='b', zorder=1, lw=2, label = 'average odd peaks')
# ax2.errorbar(dif.bincenters, q_avg_even, yerr=np.std(q_even_list, axis=0), ls='-', c='r', zorder=1, lw=2, label = 'average even peaks')
# ax2.errorbar(dif.bincenters, q_avg_odd, yerr=np.std(q_odd_list, axis = 0), ls='-', c='b', zorder=1, lw=2, label = 'average odd peaks')

ax1.set_ylabel('$q/q_0$', fontsize = 14)
ax1.legend(ncol = 2, fontsize = 10, frameon=False)
ax1.set_xlim(-2,6)
ax2.set_xlim(6,105)
ax2.set_yticklabels([]);
ax1.set_xlabel('Delay (ps)', fontsize = 14)


In [ ]:
fig, [ax1,ax2] = plt.subplots(ncols=2,nrows=1, gridspec_kw={'width_ratios': [3, 1]}, figsize = (8,6),dpi=300)# sharex=True)

fig.subplots_adjust(hspace=.05)
fig.subplots_adjust(wspace=.05)
# fig.tight_layout()

ax1.axhline(1, lw=0.5, c='k')
ax2.axhline(1, lw=0.5, c='k')
cmap = plt.get_cmap('rainbow')
# for j, bragg in enumerate(igor_bragg_list):
    
# #     sel_bragg = (dif.allBragg_indices_2[:,0] == bragg[0])&(dif.allBragg_indices_2[:,1] == bragg[1])
#     sel_bragg_sym = (dif.compact_hkl[:,0] == bragg[0])&(dif.compact_hkl[:,1] == bragg[1])
# #     print(sel_bragg_sym)
# #     fits = dif.compact[sel_bragg_sym]
# #     fit_errors = dif.compact_errors[sel_bragg_sym]
# #     fits = np.swapaxes(fits,0,1)
# #     fit_errors = np.swapaxes(fit_errors,0,1)
# #     q = dif.q_symmetrised_compact[sel_bragg_sym][0]
# #     q_error = dif.q_symmetrised_compact_errors[sel_bragg_sym][0]

# #     amp_corrected = dif.volumes_symmetrised_compact[np.where(sel_bragg_sym)[0][0]]

# #     I_tmp = amp_corrected
# #     I_tmp_error = fit_errors[:,0,0]
    
#     color = cmap((len(igor_bragg_list) - float(j+1))/(len(igor_bragg_list)+1))

#     ax1.errorbar(dif.bincenters, q/q[0:5].mean(),yerr = dt.norm_error(q, q_error), ls='-',marker='',zorder=-1, color = color, alpha = 0.5, label = bragg)
#     ax2.errorbar(dif.bincenters, q/q[0:5].mean(),yerr = dt.norm_error(q, q_error), ls='-',marker='',zorder=-1, color = color, alpha = 0.5, label = bragg)


ax1.errorbar(dif.bincenters, q_avg_even, yerr=np.std(q_even_list, axis=0), ls='-', c='r', zorder=1, lw=2, label = 'average even peaks')
ax1.errorbar(dif.bincenters, q_avg_odd, yerr=np.std(q_odd_list, axis = 0), ls='-', c='b', zorder=1, lw=2, label = 'average odd peaks')
ax2.errorbar(dif.bincenters, q_avg_even, yerr=np.std(q_even_list, axis=0), ls='-', c='r', zorder=1, lw=2, label = 'average even peaks')
ax2.errorbar(dif.bincenters, q_avg_odd, yerr=np.std(q_odd_list, axis = 0), ls='-', c='b', zorder=1, lw=2, label = 'average odd peaks')

ax1.set_ylabel('$q/q_0$', fontsize = 14)
ax1.legend(fontsize = 10, frameon=False)
ax1.set_xlim(-2,6)
ax2.set_xlim(6,105)
ax2.set_yticklabels([]);
ax1.set_xlabel('Delay (ps)', fontsize = 14)


In [ ]:
# import pickle 
# here is how I save it
with open('./lattice_expansion_data/dif_big_dataset_lattice_expansion.pkl', 'wb') as f:
    pickle.dump(dif, f)

# And here is how I would load it
# with open('saved_dictionary.pkl', 'rb') as f:
#     loaded_dict = pickle.load(f)